# Multi-Scenario MORDM

Multi-scenario MORMD is an extension of normal MORDM to better include robustness considerations within the search phase. It starts from the scenario discovery results resulting from MORDM. Next, from the experiments within this box, a set of scenarios is selected. 

There are many ways of selecting the additional scenarios. The original paper which introduced multi-scenario MORMD [Watson and Kaspzryk (2017)](https://doi.org/10.1016/j.envsoft.2016.12.001) did it in a more or less adhoc manner. [Eker and Kwakkel (2018)](https://doi.org/10.1016/j.envsoft.2018.03.029) introduced a more formal selection approach, the code of which can be found on [GitHub](https://github.com/sibeleker/MORDM---Multi-scenario-search). 

For this assignment, make an informed selection of 4 scenarios, using an approach of your choice. Motivate carefully your selection procedure. 


In [75]:
# Import all necessary packages and modules
from dps_lake_model import lake_model
from ema_workbench import (RealParameter, ScalarOutcome, Constant,
                           Model, MultiprocessingEvaluator, SequentialEvaluator, ema_logging,
                           perform_experiments, Policy, Scenario)
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from lakemodel_function import lake_problem
from ema_workbench.analysis import prim
from SALib.analyze import sobol
ema_logging.log_to_stderr(ema_logging.INFO)

import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ema_workbench.analysis import parcoords
from ema_workbench.em_framework.optimization import (HyperVolume,
                                                     EpsilonProgress)
from ema_workbench import Constraint, Policy, save_results, load_results

In [12]:
# define the lake model and its metrics
lake_model = Model('Lakemodel', function=lake_model)

lake_model.uncertainties = [RealParameter('b',0.01,0.05),
                           RealParameter('q',2.0,4.5),
                           RealParameter('mean',0.01,0.05),
                           RealParameter('stdev',0.001,0.005),
                           RealParameter('delta',0.93,0.99)]

lake_model.levers = [RealParameter('c1',-2.0,2.0),
                           RealParameter('c2',-2.0,2.0),
                           RealParameter('r1',0.0,2.0),
                           RealParameter('r2',0.0,2.0),
                           RealParameter('w1',0.0,1.0)]

lake_model.outcomes = [ScalarOutcome('max_P', kind=ScalarOutcome.MINIMIZE,
                                     expected_range=(0,5)),
                       ScalarOutcome('utility', kind=ScalarOutcome.MAXIMIZE,
                                     expected_range=(0,2)),
                       ScalarOutcome('inertia', kind=ScalarOutcome.MAXIMIZE,
                                    expected_range=(0,1)),
                       ScalarOutcome('reliability', kind=ScalarOutcome.MAXIMIZE,
                                     expected_range=(0,1))]

#convergence_metrics = [HyperVolume.from_outcomes(lake_model.outcomes),EpsilonProgress()]

In [14]:
results = load_results('results/selected_results.tar.gz')
df= results

[MainProcess/INFO] results loaded succesfully from C:\Users\ambro\PycharmProjects\EPA1361\lab_sessions\Week 5-6 - robustness and direct search\results\selected_results.tar.gz


In [20]:
results
#with MultiprocessingEvaluator(lake_model) as evaluator:
experiments, Results = results


In [21]:
experiments

,b,delta,mean,q,stdev,c1,c2,r1,r2,w1,scenario,policy,model
0,0.028993,0.973444,0.022909,2.187052,0.004248,0.251189,0.466588,0.416626,1.435007,0.986797,0,0,Lakemodel
1,0.027536,0.946723,0.035596,2.617213,0.003790,0.251189,0.466588,0.416626,1.435007,0.986797,1,0,Lakemodel
2,0.035567,0.982854,0.045683,4.175675,0.001463,0.251189,0.466588,0.416626,1.435007,0.986797,2,0,Lakemodel
3,0.047810,0.987861,0.016886,2.733429,0.001319,0.251189,0.466588,0.416626,1.435007,0.986797,4,0,Lakemodel
4,0.025888,0.944292,0.032478,4.089179,0.002928,0.251189,0.466588,0.416626,1.435007,0.986797,6,0,Lakemodel
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11979,0.044053,0.964810,0.042461,3.717552,0.001709,0.312226,0.777886,0.436954,1.601438,0.915666,995,13,Lakemodel
11980,0.035661,0.965292,0.047382,3.200350,0.003381,0.312226,0.777886,0.436954,1.601438,0.915666,996,13,Lakemodel
11981,0.036395,0.966042,0.012099,2.403512,0.003981,0.312226,0.777886,0.436954,1.601438,0.915666,997,13,Lakemodel
11982,0.038332,0.978097,0.031767,3.792047,0.003459,0.312226,0.777886,0.436954,1.601438,0.915666,998,13,Lakemodel


In [27]:
df_r = pd.DataFrame(Results)
df_r

,max_P,utility,inertia,reliability
0,36.090625,1.258633,0.9732,0.0100
1,38.041126,0.615564,0.9713,0.0100
2,30.991878,1.741175,0.9700,0.0513
3,23.045028,2.073324,0.9708,0.0300
4,39.503512,0.537296,0.9720,0.0574
...,...,...,...,...
11979,25.495845,0.945777,0.9733,0.0400
11980,31.032057,0.987308,0.9720,0.0200
11981,29.497866,0.999761,0.9710,0.0100
11982,28.625876,1.428167,0.9750,0.0492


In [79]:
df_r = pd.DataFrame(Results)

List =['max_P','utility','inertia','reliability']
List2 = []
counter = 1

for i in List:
    if i == 'max_P':    
        values = df_r[i].max()
    else: 
        values = df_r[i].min()
       
    index_e = Results.loc[df_r[i]==values].index
    print(index_e)
    line = experiments.iloc[index_e]
    List2.append(Scenario('SS'+str(counter), b=line['b'], q=line['q'], mean=line['mean'], delta=line['delta'], stdev=line['stdev']))
    
    counter += 1

Int64Index([2655], dtype='int64')
Int64Index([7063], dtype='int64')
Int64Index([    2,    12,    17,    35,    42,    50,    70,    74,    76,
               87,
            ...
            11883, 11885, 11905, 11917, 11923, 11929, 11944, 11946, 11951,
            11960],
           dtype='int64', length=786)
Int64Index([    0,     1,     5,     9,    12,    14,    16,    17,    18,
               28,
            ...
            11938, 11939, 11941, 11942, 11945, 11951, 11954, 11956, 11959,
            11981],
           dtype='int64', length=3379)


In [82]:
len(List2)
List2

[Scenario({'b': 2655    0.010134
 Name: b, dtype: float64, 'q': 2655    3.629988
 Name: q, dtype: float64, 'mean': 2655    0.049372
 Name: mean, dtype: float64, 'delta': 2655    0.981836
 Name: delta, dtype: float64, 'stdev': 2655    0.001329
 Name: stdev, dtype: float64}),
 Scenario({'b': 7063    0.035217
 Name: b, dtype: float64, 'q': 7063    4.366458
 Name: q, dtype: float64, 'mean': 7063    0.010552
 Name: mean, dtype: float64, 'delta': 7063    0.93991
 Name: delta, dtype: float64, 'stdev': 7063    0.003747
 Name: stdev, dtype: float64}),
 Scenario({'b': 2        0.035567
 12       0.041781
 17       0.033936
 35       0.016972
 42       0.012164
            ...   
 11929    0.044216
 11944    0.029645
 11946    0.028720
 11951    0.037788
 11960    0.011409
 Name: b, Length: 786, dtype: float64, 'q': 2        4.175675
 12       2.179502
 17       2.232178
 35       2.115575
 42       4.253980
            ...   
 11929    3.239548
 11944    4.436635
 11946    3.272655
 11951    2.3

## Search for each scenario

For each of the four selected scenarios, use many-objective optimization to find a pareto approximate set using the same approach as for assignment 8. Remember to check for convergence (and time permitting, seed analysis), and be careful in what epsilon values to use (not to coarse, not too small). 

Store the resulting set of pareto solutions in a smart way for subsequent analysis.


In [63]:
SS1,SS2,SS3,SS4 = List2 



SS1 = Scenario('SS1', b=0.4, q=2, mean=0.02, stdev=0.01)
SS2 = Scenario('SS2', b=0.4, q=2, mean=0.02, stdev=0.01)
SS3 = Scenario('SS3', b=0.4, q=2, mean=0.02, stdev=0.01)
SS4 = Scenario('SS4', b=0.4, q=2, mean=0.02, stdev=0.01)



,b,delta,mean,q,stdev,c1,c2,r1,r2,w1,scenario,policy,model
2655,0.010134,0.981836,0.049372,3.629988,0.001329,0.334756,0.053098,0.402136,1.619602,0.951035,99,3,Lakemodel


In [ ]:
from ema_workbench.em_framework.optimization import (HyperVolume,
                                                     EpsilonProgress)
from ema_workbench import Constraint

#specify outcomes 
lake_model.outcomes = [ScalarOutcome('max_P', kind=ScalarOutcome.MINIMIZE,
                                     expected_range=(0,5)),
                       ScalarOutcome('utility', kind=ScalarOutcome.MAXIMIZE,
                                     expected_range=(0,2)),
                       ScalarOutcome('inertia', kind=ScalarOutcome.MAXIMIZE,
                                    expected_range=(0,1)),
                       ScalarOutcome('reliability', kind=ScalarOutcome.MAXIMIZE,
                                     expected_range=(0,1))]

convergence_metrics = [HyperVolume.from_outcomes(lake_model.outcomes),
                       EpsilonProgress()]

constraints = [Constraint("max pollution", outcome_names="max_P",
                          function=lambda x:max(0, x-5))]

## Re-evaluate under deep uncertainty

Combine the pareto set of solutions found for each scenario. Next, turn each solution into a policy object. If you have a very large number of policies, you can choose to down sample your policies in some reasoned way (*e.g.*, picking min and max on each objective, slicing across the pareto front with a particular step size). As a rule of thumb, try to limit the set of policies to at most 50. 

Re-evaluate the combined set of solutions over 1000 scenarios sampled using LHS.


Calculate both the maximum regret, and the domain criterion using the values provided in [Bartholomew and Kwakkel (2020)](https://doi.org/10.1016/j.envsoft.2020.104699). Ignore the max_P objective.

visualize the results in parallel coordinate plot. 

Are there any promising compromise solutions which balance performance in both the reference scenarios as well as in terms of their robustness?
